# Scenario 4: Segmenting Crop Fields for the Austrian North
This notebook is complementary material for the walkthrough scenario **Segmenting Crop Fields for the Austrian North** used the STELAR KLMS
It is not intended to be run as a standalone notebook. It **requires access to a deployment of STELAR KLMS** and an **account** on the respective instance. 

Some of the instances used during the evaluation period of the STELAR Project are:

Internal Pilot Instance: https://klms.stelar.gr

Public Sandbox Instance: https://sandbox.stelar.gr


*If you don't have an account on the STELAR KLMS, you can create one on the respective instance. 
Kindly note that the internal pilot instance is only accessible to STELAR project members, while the public sandbox instance is open to everyone by registration.*

---
# Overview

This notebook is intended to run **Field Segmentation** on a reflectance image of tile 33UVP, having high clarity and least cloud coverage, captured by the Sentinel-2 satellite over the Austrian northern region. The resulting segmentation can be used to identify the crop fields, extract LAI, and perform further analysis.

### Prerequisites

- Fill in your accounts credentials in the block below. 
- Select datasets according to the walkthrough directions.
- Ensure you have a modern python version installed (3.9 or later).
- Install the STELAR Python SDK and any other required libraries (`pip install stelar_client --upgrade`).

### Instatiate a STELAR Client object
**Modify credentials and base URL as needed.**

In [ ]:
from stelar.client import Client, Dataset, TaskSpec, Process
from datetime import datetime

# Base URL
# Sandbox: https://sandbox.stelar.gr
# Internal Pilots: https://klms.stelar.gr

BASE_URL = "https://sandbox.stelar.gr"
USERNAME = "your_username"  # Replace with your username
PASSWORD = "your_password"  # Replace with your password

c = Client(base_url=BASE_URL, username=USERNAME, password=PASSWORD)
print(f"Connected to STELAR KLMS @ {c._base_url} as {c._username}")

### Select dataset with the reflectance image for 33UVP tile

In [ ]:
reflectance_tif_dataset = c.datasets["tile-33uvp-reflectance-sentinel-2-imagery-2021"]
print(f"Selected Dataset: {reflectance_tif_dataset.id} | {reflectance_tif_dataset.title}")
print(f"Browse the dataset at: {c._base_url}/console/v1/catalog/{reflectance_tif_dataset.id}")

### Create/Select a Workflow Process to run the matching task

In [ ]:
ORGANIZATION = "stelar-klms"

try:
    c.processes.create(**{
        "title": "Evalution Workflow for " + c._username,
        "name": "evaluation-workflow-" + c._username,
        "organization": c.organizations[ORGANIZATION]
    })
    print(f"Created new process for evaluation: {c.processes[-1].id} | {c.processes[-1].title}")
except Exception as e:
    proc = c.processes["evaluation-workflow-" + c._username]
    print(f"Using existing process for evaluation: {proc.id} | {proc.title}")

### Create a dataset to store the results of the segmentation task

In [ ]:
ORGANIZATION = "stelar-klms"

try:
    res_dset = c.datasets.create(**{
        "title": "Segmented Fields for " + c._username,
        "name": "segmented-fields-" + c._username,
        "organization": c.organizations[ORGANIZATION],
        "notes": "33UVP Tile segmented fields curated by " + c._username,
    })
    print(f"Created new dataset for segmented fields: {res_dset.id} | {res_dset.title}")
except Exception as e:
    res_dset = c.datasets["segmented-fields-" + c._username]
    print(f"Using existing dataset for segmented fields: {res_dset.id} | {res_dset.title}")

### Prepare & Run the Pesticides Matching Task

In [ ]:
# Start building the TaskSpec for Field Segmentation
t = TaskSpec(tool="field-segmentation", name="33UVP Segmentation for "+c._username)

# Define the local dataset aliases
t.d(alias='d0', dset=res_dset)

# Define the inputs
t.i(RGB=str(reflectance_tif_dataset.resources[0].id))

# Set the ResUnet model path
model_path = c.datasets['resunet-model-0-fold'].resources[0].url
t.p(model_path=model_path, sdates="2023-06-12")

# Set the outputs
timestamp= datetime.now().strftime("%Y%m%d%H%M%S")

# Define the output segmented fields to be stored inside the results dataset
t.o(segmentation_map={
    "url": f"s3://klms-bucket/evaluation/experiments/proc-" + str(proc.id) + f"/segmented_fields_{timestamp}_"+c._username+".gpkg",
    "resource":{
        "name": "Segmented Fields for " + c._username,
        "notes": "Segmented fields for 33UVP tile",
        "format": "GPKG",
        "relation": "fields"
    },
    "dataset": "d0",
})

# Define the metadata output to also be stored inside the results dataset
t.o(metadata={
    "url": f"s3://klms-bucket/evaluation/experiments/proc-" + str(proc.id) + f"/segmentation_metadata_{timestamp}_"+c._username+".json",
    "resource":{
        "name": "Segmented Fields Metadata for " + c._username,
        "notes": "Metadata for segmented fields for 33UVP tile",
        "format": "JSON",
        "relation": "log"
    },
    "dataset": "d0",
})

# Run the task using the workflow process created earlier
fsegm_task = proc.run(t)
print(f"Task {fsegm_task.id} is running. Check the status at: {c._base_url}/console/v1/task/{str(proc.id)}/{str(fsegm_task.id)}")